# Introduction to `DataFrames.jl`

---

**DataFrames.jl** is a Julia library which can be used to manipulate, clean, and query data. It was created by [Bogumił Kamiński](https://github.com/bkamins) in 2013, and is an open source project. DataFrames.jl provides a set of tools for working with tabular data in Julia.  Its design and functionality are similar to those of Pandas



### Lecture outline

---

* Construct DataFrame


* Shape, Size, Type, and Dimension of DataFrame


* Selection and Filtering


* Set index, reset index, and reindex


* Dropping Entries


* Sorting


* Column Rename, Reorder, Insertion, Deletion


* Unique Values and Value Counts


* Statistics with DataFrame

#### Reference

---

[DataFrames tutorial by Bogumił Kamiński](https://github.com/bkamins/Julia-DataFrames-Tutorial)

In [1]:
using Query
using Queryverse
using Statistics
using StatsBase
using DataFrames
using FreqTables
using Pipe: @pipe
using Random

In [2]:
# Set number of columns to be shown
ENV["COLUMNS"] = 1000

# Ser number of rows to be shown
ENV["LINES"] = 100

100

## Construct DataFrames

---

Here, we will see how to create `DataFrame` using `DataFrame()` constructor

In [3]:
# Empty DataFrame

DataFrame()

""
""


In [4]:
# DataFrame using keyword arguments

DataFrame(A=1:5, B=rand(5), C=randstring.([3, 3, 3, 3, 3]), D=5)

,A,B,C,D
,Int64,Float64,String,Int64
1,1,0.896732,Wmn,5
2,2,0.538045,XcZ,5
3,3,0.203244,BnX,5
4,4,0.439507,1LH,5
5,5,0.962897,AT7,5


In [5]:
# Create DataFrame from dictionary


data = Dict("state" => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year" => [2000, 2001, 2002, 2001, 2002, 2003],
        "population" => [1.5, 1.7, 3.6, 2.4, 2.9, 3.2])


DataFrame(data)

,population,state,year
,Float64,String,Int64
1,1.5,Ohio,2000
2,1.7,Ohio,2001
3,3.6,Ohio,2002
4,2.4,Nevada,2001
5,2.9,Nevada,2002
6,3.2,Nevada,2003


We can directly put Julia dict in `DataFrame()` constructor. The difference between this and the above approach is that here I will use `Symbol` to denote column names

In [6]:
DataFrame(:state => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    :population => [2000, 2001, 2002, 2001, 2002, 2003],
    :year => [2000, 2001, 2002, 2001, 2002, 2003])

,state,population,year
,String,Int64,Int64
1,Ohio,2000,2000
2,Ohio,2001,2001
3,Ohio,2002,2002
4,Nevada,2001,2001
5,Nevada,2002,2002
6,Nevada,2003,2003


Despite the fact that using `Symbols` is preferred as it's faster, `DataFrame()` constructor can accept the string keys in dict

In [7]:
DataFrame("state" => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    "population" => [2000, 2001, 2002, 2001, 2002, 2003],
    "year" => [2000, 2001, 2002, 2001, 2002, 2003])

,state,population,year
,String,Int64,Int64
1,Ohio,2000,2000
2,Ohio,2001,2001
3,Ohio,2002,2002
4,Nevada,2001,2001
5,Nevada,2002,2002
6,Nevada,2003,2003


Create DataFrame using array of Pairs

In [8]:
[:state => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    :population => [2000, 2001, 2002, 2001, 2002, 2003],
    :year => [2000, 2001, 2002, 2001, 2002, 2003]]

3-element Array{Pair{Symbol,Array{T,1} where T},1}:
      :state => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"]
 :population => [2000, 2001, 2002, 2001, 2002, 2003]
       :year => [2000, 2001, 2002, 2001, 2002, 2003]

In [9]:
DataFrame([:state => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    :population => [2000, 2001, 2002, 2001, 2002, 2003],
    :year => [2000, 2001, 2002, 2001, 2002, 2003]])

,state,population,year
,String,Int64,Int64
1,Ohio,2000,2000
2,Ohio,2001,2001
3,Ohio,2002,2002
4,Nevada,2001,2001
5,Nevada,2002,2002
6,Nevada,2003,2003


In [10]:
# Create DataFrame from vector of vectors

DataFrame([rand(5) for i in 1:3], [:x_1, :x_2, :x_3]) # Explicitly indicate column names

,x_1,x_2,x_3
,Float64,Float64,Float64
1,0.928456,0.739668,0.741296
2,0.755392,0.382829,0.0748615
3,0.125733,0.427613,0.608526
4,0.947184,0.692397,0.864952
5,0.65746,0.710979,0.859322


In [11]:
# Create DataFrame from vector of vectors

DataFrame([rand(5) for i in 1:3], :auto) # Automatically generate column names

,x1,x2,x3
,Float64,Float64,Float64
1,0.852818,0.0977098,0.144751
2,0.576489,0.593719,0.64337
3,0.275742,0.31446,0.810752
4,0.66125,0.80754,0.114432
5,0.485622,0.584418,0.412229


In [12]:
# Create DataFrae from vector of NamedTuple

v = [(A="a", B=1), (A="b", B="2"), (A="c", B=3), (A="d", B=4), (A="e", B=5)]

DataFrame(v)

,A,B
,String,Any
1,a,1
2,b,2
3,c,3
4,d,4
5,e,5


In [13]:
# Create DataFrame from NamedTuple of vectors

n = (A=1:5, B=randstring.([1, 1, 1, 1, 1]))

DataFrame(n)

,A,B
,Int64,String
1,1,l
2,2,G
3,3,X
4,4,e
5,5,w


In [14]:
# Create DataFrame from matrix

DataFrame(rand(5, 5), :auto)

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.582313,0.475727,0.982768,0.181704,0.317192
2,0.541174,0.974219,0.204863,0.865081,0.817155
3,0.237217,0.506213,0.936781,0.879051,0.191098
4,0.959608,0.237945,0.943663,0.429553,0.409399
5,0.272961,0.000790154,0.844619,0.124696,0.894408


## Shape, Size, Type, and Dimension of DataFrame

In [15]:
data = Dict("state" => ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year" => [2000, 2001, 2002, 2001, 2002, 2003],
        "population" => [1.5, 1.7, 3.6, 2.4, 2.9, 3.2])


df = DataFrame(data)

,population,state,year
,Float64,String,Int64
1,1.5,Ohio,2000
2,1.7,Ohio,2001
3,3.6,Ohio,2002
4,2.4,Nevada,2001
5,2.9,Nevada,2002
6,3.2,Nevada,2003


In [16]:
size(df) # Retruns number of rows and columns

(6, 3)

In [17]:
nrow(df) # Returns number of rows

6

In [18]:
ncol(df) # Retruns number of columns

3

In [19]:
ndims(df) # Number of dimensions or number of axis

2

In [20]:
eltype.(eachcol(df)) # Element types of columns

3-element Array{DataType,1}:
 Float64
 String
 Int64

In [21]:
describe(df, :eltype) # Same as above

,variable,eltype
,Symbol,DataType
1,population,Float64
2,state,String
3,year,Int64


In [22]:
# Extract column names

names(df)

3-element Array{String,1}:
 "population"
 "state"
 "year"

In [23]:
# Column names as Symbols

propertynames(df)

3-element Array{Symbol,1}:
 :population
 :state
 :year

## Selection and Filtering

In [24]:
# Let read CSV file and perform indexing, selection and filtering

df = DataFrame(load("data/admission.csv"))

first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [25]:
# Select only one column

df.SOP

df."SOP"

df[!, :SOP]

df[!, "SOP"]

400-element Array{Float64,1}:
 4.5
 4.0
 3.0
 3.5
 2.0
 4.5
 3.0
 3.0
 2.0
 3.5
 3.5
 4.0
 4.0
 4.0
 3.5
 3.5
 4.0
 4.0
 4.0
 3.5
 3.0
 3.0
 5.0
 5.0
 4.0
 4.5
 4.5
 1.5
 2.0
 1.5
 3.0
 4.0
 3.0
 4.0
 4.0
 5.0
 4.0
 1.0
 3.0
 4.0
 3.5
 2.5
 2.5
 4.5
 4.5
 5.0
 4.0
 4.5
 ⋮
 3.5
 2.5
 2.0
 3.5
 3.5
 2.5
 2.0
 4.0
 4.0
 4.5
 2.5
 3.5
 4.5
 3.5
 1.0
 2.0
 2.0
 2.5
 3.5
 4.5
 3.0
 2.0
 2.0
 2.5
 1.5
 2.0
 2.5
 3.5
 3.0
 4.5
 3.0
 5.0
 5.0
 2.5
 2.0
 1.5
 3.5
 2.0
 2.0
 4.0
 3.0
 4.5
 3.5
 3.0
 5.0
 3.5
 5.0

In [26]:
# Select two or more columns

df[!, [:SOP, :LOR]]

df[!, ["SOP", "LOR"]]

,SOP,LOR
,Float64,Float64
1,4.5,4.5
2,4.0,4.5
3,3.0,3.5
4,3.5,2.5
5,2.0,3.0
6,4.5,3.0
7,3.0,4.0
8,3.0,4.0
9,2.0,1.5


### Selection

---

`DataFrames.jl` does not have dedicated methods for column and row selection such as Pandas `loc` and `iloc`. Selection is performed using by square brackets.

In [27]:
first(df) # Select first row by default

first(df, 3) # Select first 3 rows


last(df) # Select last row by default

last(df, 3) # Select last 3 rows

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,398,330,116,4,5.0,4.5,9.45,1,0.91
2,399,312,103,3,3.5,4.0,8.78,0,0.67
3,400,333,117,4,5.0,4.0,9.66,1,0.95


Selection using axis labels. Purely label-location based indexer. Analogue of Pandas `loc`

In [28]:
df[:, :] # Select all rows and columns

df[:, "SOP"] # Select all rows and one column

df[1:3, "SOP"] # Select first three entries of one column

df[1:10, ["GRE Score", "TOEFL Score"]] # Select first 10 entries of two or more columns

df[5:10, Between(:SOP, :Research)] # Select range of rows and range of columns

,SOP,LOR,CGPA,Research
,Float64,Float64,Float64,Int64
1,2.0,3.0,8.21,0
2,4.5,3.0,9.34,1
3,3.0,4.0,8.2,1
4,3.0,4.0,7.9,0
5,2.0,1.5,8.0,0
6,3.5,3.0,8.6,0


Selecting using integers. Purely integer-location based indexing. Analogue of Pandas `iloc`

In [29]:
df[1, 1] # Select first element of the first column

df[2, :] # Select only one row

df[1:5, :] # Select range of rows

df[1:5, 1] # Select 5 rows of the first columns

df[1:10, 2:5] # Select row as well as column range

df[[1, 2, 3], [1, 2, 3]] # Select particular rows and columns

,Serial No.,GRE Score,TOEFL Score
,Int64,Int64,Int64
1,1,337,118
2,2,324,107
3,3,316,104


### Boolean Indexing

---

Moreover, we can perform row and column selection based on boolean indexing. This is more like to select rows or columns which satisfy pre-defined condition(s).


Boolean indexing means to use boolean series, only True and/or False to select rows or columns.

In [30]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [31]:
df[!, "GRE Score"] .> 325 # Returns boolean Series. True when condition is True, otherwise False

df[df[!, "GRE Score"] .> 325, :] # Returns corresponding DataFrame. Where True

df[.!(df[!, "GRE Score"] .> 325), :] # Returns DataFrame where False

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.0,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.8
4,5,314,103,2,2.0,3.0,8.21,0,0.65
5,7,321,109,3,3.0,4.0,8.2,1,0.75
6,8,308,101,2,3.0,4.0,7.9,0,0.68
7,9,302,102,1,2.0,1.5,8.0,0,0.5
8,10,323,108,3,3.5,3.0,8.6,0,0.45
9,11,325,106,3,3.5,4.0,8.4,1,0.52


In [32]:
df[.!(df[!, "GRE Score"] .> 325), [:SOP, :LOR]]

,SOP,LOR
,Float64,Float64
1,4.0,4.5
2,3.0,3.5
3,3.5,2.5
4,2.0,3.0
5,3.0,4.0
6,3.0,4.0
7,2.0,1.5
8,3.5,3.0
9,3.5,4.0


#### Chained Conditionals

With the chained conditionals, we can filter DataFrame based on several conditions chained by using logic operators.

In [33]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [34]:
condition_one = (df[!, "GRE Score"] .> 325)

condition_two = (df[!, "Research"] .== 1)

condition_three = (df[!, "CGPA"] .> 8.00)

400-element BitArray{1}:
 1
 1
 0
 1
 1
 1
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

**Note**, I'm broadcasting the bitwise `AND` operator

In [35]:
df[condition_one .& condition_two, :] # Only select rows where BOTH conditions satisfy

df[condition_one .| (condition_two .& condition_three), :] # Only select rows where ONE of the condition satisfy

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,4,322,110,3,3.5,2.5,8.67,1,0.8
4,6,330,115,5,4.5,3.0,9.34,1,0.9
5,7,321,109,3,3.0,4.0,8.2,1,0.75
6,11,325,106,3,3.5,4.0,8.4,1,0.52
7,12,327,111,4,4.0,4.5,9.0,1,0.84
8,13,328,112,4,4.0,4.5,9.1,1,0.78
9,15,311,104,3,3.5,2.0,8.2,1,0.61


To have similar functionality of Pandas `query()` method, we need to use `Query.jl` library.

In [36]:
df |> @filter(_.SOP .> _.LOR) |> DataFrame # Select rows where "SOP" is more than "LOR"

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,4,322,110,3,3.5,2.5,8.67,1,0.8
2,6,330,115,5,4.5,3.0,9.34,1,0.9
3,9,302,102,1,2.0,1.5,8.0,0,0.5
4,10,323,108,3,3.5,3.0,8.6,0,0.45
5,14,307,109,3,4.0,3.0,8.0,1,0.62
6,15,311,104,3,3.5,2.0,8.2,1,0.61
7,16,314,105,3,3.5,2.5,8.3,0,0.54
8,17,317,107,3,4.0,3.0,8.7,0,0.66
9,18,319,106,3,4.0,3.0,8.0,1,0.65


In [37]:
df |> @filter((_.SOP .> _.LOR) .& (_.CGPA .> 9)) |> DataFrame # Combine conditionals with AND

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,6,330,115,5,4.5,3.0,9.34,1,0.9
2,24,334,119,5,5.0,4.5,9.7,1,0.95
3,25,336,119,5,4.0,3.5,9.8,1,0.97
4,44,332,117,4,4.5,4.0,9.1,0,0.87
5,45,326,113,5,4.5,4.0,9.4,1,0.91
6,46,322,110,5,5.0,4.0,9.1,1,0.88
7,48,339,119,5,4.5,4.0,9.7,0,0.89
8,70,328,115,4,4.5,4.0,9.16,1,0.78
9,74,314,108,4,4.5,4.0,9.04,1,0.84


## Set index, reset index, and reindex


---

unfortunately, these functionalities are not yet implemented in `DataFrames.jl` package. [See the following thread](https://stackoverflow.com/questions/64689023/is-it-possible-to-set-a-chosen-column-as-index-in-a-julia-dataframe)


**The only possibility is to use `indexmap` function to map unique values of a column to its corresponding index.**

In [38]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [39]:
indexmap(df[!, :SOP]) # Returns unique values and its index

Dict{Float64,Int64} with 9 entries:
  4.0 => 2
  2.0 => 5
  3.0 => 3
  2.5 => 42
  3.5 => 4
  1.5 => 28
  4.5 => 1
  5.0 => 23
  1.0 => 38

## Dropping Entries

---

We can drop entries from rows and/or columns.

In [40]:
last(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,396,324,110,3,3.5,3.5,9.04,1,0.82
2,397,325,107,3,3.0,3.5,9.11,1,0.84
3,398,330,116,4,5.0,4.5,9.45,1,0.91
4,399,312,103,3,3.5,4.0,8.78,0,0.67
5,400,333,117,4,5.0,4.0,9.66,1,0.95


In [41]:
delete!(df, 400) # Mutationg function. Does in=place removal

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65
6,6,330,115,5,4.5,3.0,9.34,1,0.9
7,7,321,109,3,3.0,4.0,8.2,1,0.75
8,8,308,101,2,3.0,4.0,7.9,0,0.68
9,9,302,102,1,2.0,1.5,8.0,0,0.5


In [42]:
last(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,395,329,111,4,4.5,4.0,9.23,1,0.89
2,396,324,110,3,3.5,3.5,9.04,1,0.82
3,397,325,107,3,3.0,3.5,9.11,1,0.84
4,398,330,116,4,5.0,4.5,9.45,1,0.91
5,399,312,103,3,3.5,4.0,8.78,0,0.67


In [43]:
select(df, Not(:SOP)) # Remove one column

select(df, Not([:SOP, :LOR])) # Remove two or more columns

,Serial No.,GRE Score,TOEFL Score,University Rating,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Int64,Float64
1,1,337,118,4,9.65,1,0.92
2,2,324,107,4,8.87,1,0.76
3,3,316,104,3,8.0,1,0.72
4,4,322,110,3,8.67,1,0.8
5,5,314,103,2,8.21,0,0.65
6,6,330,115,5,9.34,1,0.9
7,7,321,109,3,8.2,1,0.75
8,8,308,101,2,7.9,0,0.68
9,9,302,102,1,8.0,0,0.5


## Sorting

---

Sorting is an important operation. We can sort DataFrame by one or several columns and with different sorting order, ascending or descending.

In [44]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [45]:
sort(df, :SOP, rev=false) # Sort by column in ascending order

sort(df, :SOP, rev=true) # Sort by column in descending order

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,23,328,116,5,5.0,5.0,9.5,1,0.94
2,24,334,119,5,5.0,4.5,9.7,1,0.95
3,36,320,110,5,5.0,5.0,9.2,1,0.88
4,46,322,110,5,5.0,4.0,9.1,1,0.88
5,71,332,118,5,5.0,5.0,9.64,1,0.94
6,72,336,112,5,5.0,5.0,9.76,1,0.96
7,73,321,111,5,5.0,5.0,9.45,1,0.93
8,82,340,120,4,5.0,5.0,9.5,1,0.96
9,83,320,110,5,5.0,4.5,9.22,1,0.92


In [46]:
sort(df, ["CGPA", "Research"], rev=true)

sort(df, [order("CGPA", rev=false), order("Research", rev=true)]) # Sort by two columns. Ascending and descending

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,59,300,99,1,3.0,2.0,6.8,1,0.36
2,29,295,93,1,2.0,2.0,7.2,0,0.46
3,349,302,99,1,2.0,2.0,7.25,0,0.57
4,119,296,99,2,3.0,3.5,7.28,0,0.47
5,30,310,99,2,1.5,2.0,7.3,0,0.54
6,345,295,96,2,1.5,2.0,7.34,0,0.47
7,348,299,94,1,1.0,1.0,7.34,0,0.42
8,80,294,93,1,1.5,2.0,7.36,0,0.46
9,57,316,102,3,2.0,3.0,7.4,0,0.64


## Column Rename, Reorder, Insertion, Deletion

---

DataFrame is dynamic, meaning that we can add and remove columns, as well as reorder or rename them.

### Rename Columns


In [47]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [48]:
# We have to indicate mapper. Which column we want to rename and new name of it

rename(df, "GRE Score"=> "GRE", "TOEFL Score"=> "TOEFL")

,Serial No.,GRE,TOEFL,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65
6,6,330,115,5,4.5,3.0,9.34,1,0.9
7,7,321,109,3,3.0,4.0,8.2,1,0.75
8,8,308,101,2,3.0,4.0,7.9,0,0.68
9,9,302,102,1,2.0,1.5,8.0,0,0.5


### Reorder Columns

---

Sometimes we may need to just re-order the columns. There are a few ways to do it.

In [49]:
using Random

Random.seed!(1234);

In [50]:
df[:, shuffle(names(df))] # Randomly shuffle columns

,LOR,Serial No.,GRE Score,TOEFL Score,Chance of Admit,SOP,CGPA,University Rating,Research
,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64
1,4.5,1,337,118,0.92,4.5,9.65,4,1
2,4.5,2,324,107,0.76,4.0,8.87,4,1
3,3.5,3,316,104,0.72,3.0,8.0,3,1
4,2.5,4,322,110,0.8,3.5,8.67,3,1
5,3.0,5,314,103,0.65,2.0,8.21,2,0
6,3.0,6,330,115,0.9,4.5,9.34,5,1
7,4.0,7,321,109,0.75,3.0,8.2,3,1
8,4.0,8,308,101,0.68,3.0,7.9,2,0
9,1.5,9,302,102,0.5,2.0,8.0,1,0


In [51]:
new_cols = ["GRE Score", "TOEFL Score", "CGPA", "LOR", "SOP",
            "Serial No.", "Chance of Admit ", "Research", "University Rating"]

9-element Array{String,1}:
 "GRE Score"
 "TOEFL Score"
 "CGPA"
 "LOR"
 "SOP"
 "Serial No."
 "Chance of Admit "
 "Research"
 "University Rating"

In [52]:
select(df, new_cols) # Reorder columns by changing column names order and then perform selection

,GRE Score,TOEFL Score,CGPA,LOR,SOP,Serial No.,Chance of Admit,Research,University Rating
,Int64,Int64,Float64,Float64,Float64,Int64,Float64,Int64,Int64
1,337,118,9.65,4.5,4.5,1,0.92,1,4
2,324,107,8.87,4.5,4.0,2,0.76,1,4
3,316,104,8.0,3.5,3.0,3,0.72,1,3
4,322,110,8.67,2.5,3.5,4,0.8,1,3
5,314,103,8.21,3.0,2.0,5,0.65,0,2
6,330,115,9.34,3.0,4.5,6,0.9,1,5
7,321,109,8.2,4.0,3.0,7,0.75,1,3
8,308,101,7.9,4.0,3.0,8,0.68,0,2
9,302,102,8.0,1.5,2.0,9,0.5,0,1


### Insert new columns

In [53]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [54]:
df[!, "A"] = df[!, "CGPA"] * 100 # Add new column

first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,A
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92,965.0
2,2,324,107,4,4.0,4.5,8.87,1,0.76,887.0
3,3,316,104,3,3.0,3.5,8.0,1,0.72,800.0
4,4,322,110,3,3.5,2.5,8.67,1,0.8,867.0
5,5,314,103,2,2.0,3.0,8.21,0,0.65,821.0


The above technique can add new column only at the end of the DataFrame. If we want to add new column at specified position, we can use `insertcols!()` function. Note that `insertcols!()` is in-place function.

In [55]:
insertcols!(df, 1, "B"=> 1) # Add new columns named "B" containing only 1's at the first place

,B,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,A
,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64,Float64
1,1,1,337,118,4,4.5,4.5,9.65,1,0.92,965.0
2,1,2,324,107,4,4.0,4.5,8.87,1,0.76,887.0
3,1,3,316,104,3,3.0,3.5,8.0,1,0.72,800.0
4,1,4,322,110,3,3.5,2.5,8.67,1,0.8,867.0
5,1,5,314,103,2,2.0,3.0,8.21,0,0.65,821.0
6,1,6,330,115,5,4.5,3.0,9.34,1,0.9,934.0
7,1,7,321,109,3,3.0,4.0,8.2,1,0.75,820.0
8,1,8,308,101,2,3.0,4.0,7.9,0,0.68,790.0
9,1,9,302,102,1,2.0,1.5,8.0,0,0.5,800.0


### Remove columns


---

In Julia there is no dedicated function for removing columns. So, we need to use `select!()` with `Not()` to remove columns.

In [56]:
select!(df, Not(["A", "B"]))

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65
6,6,330,115,5,4.5,3.0,9.34,1,0.9
7,7,321,109,3,3.0,4.0,8.2,1,0.75
8,8,308,101,2,3.0,4.0,7.9,0,0.68
9,9,302,102,1,2.0,1.5,8.0,0,0.5


## Unique Values and Value Counts

---

As the header suggests, we can count the number of unique values in a column, as well as count how many times a certain value occur.

### Unique Values

---

We can find the number of unique values in a column by using `unique()` function.

In [57]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [58]:
unique(df[:, "University Rating"]) # Unique values for University Rating

5-element Array{Int64,1}:
 4
 3
 2
 5
 1

### Value Counts

---

Value counts represent the operation when we want to count the number of unique values in a column. In short, calculate value frequencies

In [59]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [60]:
counts(df[:, "Research"]) # Counts unique values

2-element Array{Int64,1}:
 181
 218

In [61]:
df[:, "Research"] |> freqtable # We have 219 ones and 181 zeros, totaling to 400

2-element Named Array{Int64,1}
Dim1  │ 
──────┼────
0     │ 181
1     │ 218

In [62]:
countmap(df[:, "Research"]) # Same as above

Dict{Int64,Int64} with 2 entries:
  0 => 181
  1 => 218

In [63]:
df[:, "Research"] |> freqtable |> prop # Proportions

2-element Named Array{Float64,1}
Dim1  │ 
──────┼─────────
0     │ 0.453634
1     │ 0.546366

## Statistics with DataFrame

In [64]:
first(df, 5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,324,107,4,4.0,4.5,8.87,1,0.76
3,3,316,104,3,3.0,3.5,8.0,1,0.72
4,4,322,110,3,3.5,2.5,8.67,1,0.8
5,5,314,103,2,2.0,3.0,8.21,0,0.65


In [65]:
describe(df) # Summary statistics

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Serial No.,200.0,1,200.0,399,0,Int64
2,GRE Score,316.767,290,317.0,340,0,Int64
3,TOEFL Score,107.386,92,107.0,120,0,Int64
4,University Rating,3.08521,1,3.0,5,0,Int64
5,SOP,3.39599,1.0,3.5,5.0,0,Float64
6,LOR,3.45113,1.0,3.5,5.0,0,Float64
7,CGPA,8.59627,6.8,8.6,9.92,0,Float64
8,Research,0.546366,0,1.0,1,0,Int64
9,Chance of Admit,0.723784,0.34,0.73,0.97,0,Float64


### Covariance and Correlation

---

These statistics compared to above-mentioned ones requires pairs of values or at least two Series to give meaningful results. Let use our DataFrame to calculate covariance and correlation between some columns.

### Covariance

In [66]:
DataFrame(cov(Matrix(df)), names(df)) # Compute pairwise covariances among the series in the DataFrame

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,13300.0,-137.832,-108.887,-22.9874,-20.2852,-9.46231,-3.68548,-3.87437,0.586382
2,-137.832,131.315,57.9721,8.76363,7.03223,5.73984,5.67079,3.30857,1.30737
3,-108.887,57.9721,36.6999,4.81879,3.99251,3.09052,2.98024,1.47452,0.68145
4,-22.9874,8.76363,4.81879,1.3093,0.844312,0.678798,0.507957,0.254833,0.115782
5,-20.2852,7.03223,3.99251,0.844312,1.00988,0.659477,0.42799,0.221543,0.0963619
6,-9.46231,5.73984,3.09052,0.678798,0.659477,0.808535,0.358523,0.177523,0.0857382
7,-3.68548,5.67079,2.98024,0.507957,0.42799,0.358523,0.353652,0.154206,0.0738483
8,-3.87437,3.30857,1.47452,0.254833,0.221543,0.177523,0.154206,0.248473,0.0391583
9,0.586382,1.30737,0.68145,0.115782,0.0963619,0.0857382,0.0738483,0.0391583,0.0202603


### Correlation

In [67]:
DataFrame(cor(Matrix(df)), names(df)) # Compute pairwise correlation among the series in the DataFrame

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,-0.104296,-0.155854,-0.174199,-0.175032,-0.0912477,-0.0537378,-0.0673963,0.0357217
2,-0.104296,1.0,0.835083,0.668355,0.610661,0.557049,0.832144,0.579221,0.801527
3,-0.155854,0.835083,1.0,0.695161,0.655809,0.567348,0.827239,0.488292,0.790277
4,-0.174199,0.668355,0.695161,1.0,0.734256,0.659737,0.746482,0.446782,0.710885
5,-0.175032,0.610661,0.655809,0.734256,1.0,0.729818,0.71616,0.442266,0.67367
6,-0.0912477,0.557049,0.567348,0.659737,0.729818,1.0,0.670471,0.396064,0.669888
7,-0.0537378,0.832144,0.827239,0.746482,0.71616,0.670471,1.0,0.520205,0.87243
8,-0.0673963,0.579221,0.488292,0.446782,0.442266,0.396064,0.520205,1.0,0.551901
9,0.0357217,0.801527,0.790277,0.710885,0.67367,0.669888,0.87243,0.551901,1.0


We have the same situation as in case of covariance, or we see that above and below main diagonal there are the same elements. **Correlation is scaled form of the Covariance**, hence this result was expected too.

> **Covariance is a measure of correlation and it indicates direction of linear relationship between two variables.**


> **Correlation is scaled Covariance or the values are standardized. Correlation measures both the strength and direction of the linear relationship between two variables**

# Summary

---

In this class we've covered quite broad range of `DataFrames.jl` functionality. From the simplest operation of data creation or reading to performing statistical operations on DataFrames. In the next classes we dig deeper.